In [1]:
%load_ext autoreload
%autoreload 2

import sys
sys.path.append('..')  # enable import from src/

In [2]:
import pickle
from pathlib import Path

import gurobipy
from gurobipy import GRB
import dgl
import numpy as np
import torch

from src.data import get_model, get_soc, load_instance

In [3]:
instances = sorted(list(Path('../data/raw').glob('97_9*.jl')))
instances

[PosixPath('../data/raw/97_9.jl'),
 PosixPath('../data/raw/97_9_1.jl'),
 PosixPath('../data/raw/97_9_2.jl'),
 PosixPath('../data/raw/97_9_3.jl'),
 PosixPath('../data/raw/97_9_4.jl'),
 PosixPath('../data/raw/97_9_5.jl')]

In [4]:
instance = load_instance(instances[0])

model = get_model(list(range(9)), instance, coupling=True)

model.optimize()

model.ObjVal

Set parameter Username
Academic license - for non-commercial use only - expires 2024-02-10

Interrupt request received


1143.0

In [ ]:
X = np.array([v.X for v in model.getVars()])
soc_vars_mask = np.array(['soc' in v.getAttr(GRB.Attr.VarName) for v in model.getVars()])
soc = X[soc_vars_mask]
X = X[~soc_vars_mask]

# check that get_soc is working
soc_hat = get_soc(torch.from_numpy(X[None,:]), instance)
torch.isclose(soc_hat[0][1:], torch.from_numpy(soc)).all()

(1746,)

In [6]:
instance = load_instance(instances[3])

model = get_model(list(range(9)), instance, coupling=True)

with open('../97_9_opts.pkl', 'rb') as f:
    opts = pickle.load(f)

opts[instances[3].name]['sol'].shape

(1746,)

In [7]:
from src.dataset import InstanceEarlyFixingDataset
from src.net import InstanceGCN

net = InstanceGCN(2, readout_op=None)

ds = InstanceEarlyFixingDataset(
    [load_instance(i) for i in instances],
    [opts[i.name]['sol'] for i in instances],
    samples_per_problem=10,
)

g1, y1 = ds[0]
g2, y2 = ds[13]

g = dgl.batch((g1, g2))

net(g).shape

torch.Size([2, 1746])